In [1]:
#多通道卷积先相应通道做卷积在相加
from d2l import torch as d2l
import torch

def corr2d_multi_in(X, K):
    # First, iterate through the 0th dimension (channel dimension) of `X` and
    # `K`. Then, add them together
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [2]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [3]:
#Multiple Output Channels
# the shape of convolution kernel is Co * Ci * Kh * Kw 相当于多个卷积核每个卷积核对应一张特征图
def corr2d_multi_in_out(X, K):

    # return torch.stack([corr2d_multi_in(X, K) for k in K], 0)
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [4]:
K = torch.stack((K, K+1, K+2),0) #0代表堆叠方向
K.shape

torch.Size([3, 2, 2, 2])

In [5]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [6]:
# 1*1的卷积层 没有识别高度和宽度之间的相邻元素的相互作用，去组合每个通道之间的特征
# X 2x3x3 K 2x1x1
# 改成全连接 然后相乘
def corr2d_muti_in_out_1x1(X, K):
    c_i, h, w =X.shape # 2,3,3
    c_o = K.shape[0] #卷积核的个数 3
    X = X.reshape((c_i,h * w))
    K = K.reshape((c_o,c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [7]:
X = torch.normal(0, 1,(3, 3, 3))
K = torch.normal(0,1,(2,3,1,1))

Y1 = corr2d_muti_in_out_1x1(X ,K)
Y2 = corr2d_multi_in_out(X, K)
assert float(d2l.reduce_sum(torch.abs(Y1 - Y2 ))) < 1e-6

In [8]:
Y1.shape

torch.Size([2, 3, 3])